<a href="https://colab.research.google.com/github/goelnikhils-lgtm/languagemodels/blob/main/Autonomous_Agent_LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Autonomus agent built using LangGraph

In [ ]:
!pip install uv
!pip install langchain langgraph langchain_community
!pip install python-dotenv

In [ ]:
import os
from typing import Dict , List , Tuple , Any , Optional , TypedDict , Annotated
#from langchain_openai import ChatOpenAI

from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph import StateGraph, END
from google.colab import userdata

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms import HuggingFacePipeline

from langchain_community.chat_models import ChatHuggingFace
from langchain_community.llms import HuggingFaceHub

# Initialize HuggingFaceHub with your Hugging Face API token and the model repo ID
# Set the OpenAI API key from Colab secrets
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get("HUGGINGFACEHUB_API_TOKEN")
llm = HuggingFaceHub(repo_id="meta-llama/Llama-3.1-8B-Instruct")

# Wrap it with ChatHuggingFace
llm = ChatHuggingFace(llm=llm)



#model_name = "meta-llama/Llama-2-7b-chat-hf" # Example: Replace with your desired Llama model
#model = AutoModelForCausalLM.from_pretrained(model_name)
#pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100)
#llm = HuggingFacePipeline(pipeline=pipe)


#llm = ChatOpenAI(model="meta-llama/Llama-3.1-8B-Instruct", temperature=0)
#define the agent state
class AgentState(TypedDict):
    """State of the agent"""
    question: str
    sub__questions: List[str]
    research_findings: List[str]
    current_sub_question: Optional[str]
    final_answer: Optional[str]
    evaluation: Optional[str]
    messages: List[Any]
#intialize the state graph
graph = StateGraph(AgentState)

In [ ]:
import os
from typing import Dict , List , Tuple , Any , Optional , TypedDict , Annotated
#from langchain_openai import ChatOpenAI

from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph import StateGraph, END
from google.colab import userdata

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms import HuggingFacePipeline

from langchain_community.chat_models import ChatHuggingFace
from langchain_community.llms import HuggingFaceHub

# Initialize HuggingFaceHub with your Hugging Face API token and the model repo ID
# Set the OpenAI API key from Colab secrets
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get("HUGGINGFACEHUB_API_TOKEN")
# llm = HuggingFaceHub(repo_id="meta-llama/Llama-3.1-8B-Instruct")

# Wrap it with ChatHuggingFace
#llm = ChatHuggingFace(llm=llm)



model_name = "meta-llama/Llama-3.1-8B-Instruct" # Example: Replace with your desired Llama model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100)
llm = HuggingFacePipeline(pipeline=pipe)


#llm = ChatOpenAI(model="meta-llama/Llama-3.1-8B-Instruct", temperature=0)
#define the agent state
class AgentState(TypedDict):
    """State of the agent"""
    question: str
    sub_questions: List[str]
    research_findings: Dict[str, str]
    current_sub_question: Optional[str]
    final_answer: Optional[str]
    evaluation: Optional[str]
    messages: List[Any]
#intialize the state graph
graph = StateGraph(AgentState)

#define the breakdown node - divides the main question into sub-questions
def breakdown_question(state: AgentState) -> AgentState:
  """Breakdown the main question into sub-questions"""
  messages = [
      SystemMessage(content = "You are a research expert skilled at breaking complex questions into specific sub-questions. Your job is to analyze provided research questions and break it down into clear 3-5 sub-questions."),
      HumanMessage(content = f"Please break down the following research question into 3-5 sub questions: \n\n{state['question']}\n\n. Format your response as a list of sub-questions one per line , withour numbering or bullets")
      ]
  response = llm.invoke(messages)
  sub_questions = [q.strip() for q in response.split('\n') if q.strip()]
  return {
      ** state,
      "sub_questions":sub_questions,
      "research_findings":{},
      "messages":state["messages"] + [
          HumanMessage(content = f"Breaking down the question:{state['question']}"),
          AIMessage(content = f"I have broken down your question into these sub-questions:\n{response}")
          ]
      }
#define the research node - researches the current sub-question
def research_sub_question(state:AgentState) ->AgentState:
    """Research the current sub-question"""
    current_sub_question = state["sub_questions"][0] if not state ["current_sub_question"] else state["current_sub_question"]
    messages = [
      SystemMessage(content = "You are a thorough research assistant. Your taks is to provide adetailed answer to a specific research question using your knowledge"),
      HumanMessage(content = f"Please research the following question and provide a detailed answer: \n\n{current_sub_question}")
      ]
    response = llm.invoke(messages)
    research_findings = state["research_findings"].copy()
    research_findings[current_sub_question] = response

    return {
      ** state,
      "current_sub_question":current_sub_question,
      "research_findings":research_findings,
      "messages":state["messages"] + [
          HumanMessage(content = f"Researching:{current_sub_question}"),
          AIMessage(content = f"Research Findings:\n{response[:100]}...")
          ]
      }
#Define the next question selector - determines if we should research another question or synthesize
def select_next_steps(state:AgentState) ->str:
  """Determine whether to research the next question or move to synthesis"""
  researched_questions = set(state["research_findings"].keys())
  remanining_questions = [q for q in state["sub_questions"] if q not in researched_questions]
  if remanining_questions:
    state["current_sub_question"] = remanining_questions[0]
    return "research"
  else:
    return "synthesize"

#Define the synthesis node - combines research findings into a cohesive manner
def synthesize_findings(state:AgentState) ->AgentState:
  """Synthesize all research findings into a comprehensive answer"""
  findings_text = "\n".join([f"Sub-question: {q}:\n Findings: {f}" for q, f in state["research_findings"].items()])
  messages = [
         SystemMessage(content = "You are a research synthesis expert. Your task is to combine separate research findings into a cohesive,  well-structured answer to the original question."),
         HumanMessage(content =f"Original question:{state['question']}\n\n Research Findings:\n{findings_text}\n\n Please synthesize these findings into a comprehensive answer to the original question.")
         ]
  response = llm.invoke(messages)
  return{
         **state,
         "final_answer":response,
         "messages":state["messages"] +[
             HumanMessage(content="Synthesizing rsearch findings"),
             AIMessage(content=f"I 've synthesized the findings into a comprehensive answer:\n{response[:100]}...")
         ]
     }

#Define the evaluation node - assess the quality of the final answer
def evaluate_answer(state:AgentState) ->AgentState:
  """Evaluate the quality of the synthesized answer"""
  messages = [
         SystemMessage(content = "You are a critical evaluator of research answers. Your task is to identify any gaps , logical flows, or areas where the answer could be improved"),
         HumanMessage(content =f"Original question:{state['question']}\n\n Synthesized answer:{state['final_answer']}\n\n Please evauuate this answer , identifying any weakness or areas of improvement.")
         ]
  response = llm.invoke(messages)
  return{
         **state,
         "evaluation":response,
         "messages":state["messages"] +[
             HumanMessage(content="Evaluating the answer quality"),
             AIMessage(content=f"Evaluation:\n{response}")
         ]
     }
#define the refinement decision node - determines if the answer needs refinement
def needs_refinement(state:AgentState) ->str:
  """Determine if the answer needs refinement based on evaluation"""
  if not state["evaluation"] or "excellent" in state["evaluation"].lower() or "adequate" in state["evaluation"].lower():
    return "complete"
  else:
    return "refine"

#Define the evaluation node - assess the quality of the final answer
def refine_answer(state:AgentState) ->AgentState:
  """Refine the answer based on evaluation feedback"""
  messages = [
         SystemMessage(content = "You are a expert at refining research answers. Your task is to improve an answer based on specific evaluation feedback"),
         HumanMessage(content =f"Original question:{state['question']}\n\n Current answer:{state['final_answer']}\n\n Evaluation feedback:{state['evaluation']}\n\n Please provide an improved version of the answer that addresses the weakness identified in the evaluation.")
         ]
  response = llm.invoke(messages)
  return{
         **state,
         "final_answer":response,
         "messages":state["messages"] +[
             HumanMessage(content="Refining the answer based on evaluation..."),
             AIMessage(content=f"Refined answer :\n{response[:100]}...")
         ]
     }

In [ ]:
# Initialize the state graph again to avoid "Node already present" errors on re-execution
graph = StateGraph(AgentState)

#create the agent execution graph
graph.add_node("breakdown",breakdown_question)
graph.add_node("research",research_sub_question)
graph.add_node("synthesize",synthesize_findings)
graph.add_node("evaluate",evaluate_answer)
graph.add_node("refine",refine_answer)

#define the edges - the flow of execution between nodes
graph.add_edge("breakdown","research")
graph.add_conditional_edges(
    "research",
    select_next_steps,
    {
        "research": "research", #continue researching
        "synthesize":"synthesize" #move to Synthesis
    }
)
graph.add_edge("synthesize","evaluate")
graph.add_conditional_edges(
    "evaluate",
    needs_refinement,
    {
        "refine": "refine", #continue researching
        "complete":END #Finish execution
    }
)
graph.add_edge("refine","evaluate")
#set the entry point
graph.set_entry_point("breakdown")
#Compile the graph into an executable
research_agent = graph.compile()

#user interface
def run_research_agent(question:str):
  """Run the research agent on a given question"""
  initial_state={
      "question":question,
      "sub_questions":[],
      "research_findings":{},
      "current_sub_question":None,
      "final_answer":None,
      "evaluation":None,
      "messages":[]
  }
  print(f"Researching:{question}\n")
  print(f"Start the research process...\n")

  final_state = research_agent.invoke(initial_state)

  print("\n=== Sub-Questions===")
  for i , q in enumerate(final_state["sub_questions"],1):
    print(f"{i+1}.{q}")

  print("\n=== Final Answer===")
  print(final_state["final_answer"])

  print("\n=== Evaluation===")
  print(final_state["evaluation"])
  return final_state

#Example usage

if __name__ == "__main__":
  question = "What are the enviornmental and economic impacts of switching to renewable energy sources , what are the main challenges to widespread adoption?"
  run_research_agent(question)